In [1]:
import sqlalchemy
import json
import pandas as pd
import datetime
import os
import numpy as np
import gc

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
from sklearn import metrics
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

print(sklearn.__version__)

In [ ]:
class SM_Logistic_Model:
    
    # 1
    def __init__(self,
                 config_path="/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/config.json"):
        self.output_folder=""
        
        self.high_date=""
        self.username=""
        self.password=""
        self.database=""
        self.recent_n_month=""

        self.table_df_1=""
        self.table_2_1=""
        self.table_2_2=""
        self.table_0_train=""
        self.table_0_test=""
        
        self.iv_end_date=""
        self.iv_start_date=""
        self.dv_end_date=""
        self.dv_start_date=""        
        
        self.df_train_all=pd.DataFrame()
        
        self.df_date_range=pd.DataFrame()
        self.output_folder=""
        self.output_path=""
        self.df_department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",
                                              sep="|").drop_duplicates() 
    # 2 config
    def config(self):
        with open('./config.json', 'rb') as f:
            data = json.load(f)
        self.high_date=data['pos_end_date']
        self.username=data['username']
        self.password=data['password']
        self.database=data['database']
        self.recent_n_month=data['recent_n_month']
        
        self.table_df_1=""
        self.table_2_1=""
        self.table_2_2=""
        self.table_0_train=""
        self.table_0_test=""
        
        self.iv_end_date=""
        self.iv_start_date=""
        self.dv_end_date=""
        self.dv_start_date=""  
        
        
               
    
        
    # 2     
    def remove_low_variate_iv(self,r_variance=0.98):
        threshold_variance_iv=r_variance*(1-r_variance)
        selector = VarianceThreshold(threshold=threshold_variance_iv)
        df_redused_X=selector.fit_transform(self.df_X)
        print("df_X reduced to the shape due to %s variante"%(str(r_variance)),df_redused_X.shape)

        indices = [i for i, x in enumerate(list(selector.get_support())) if x == True]
        self.X_features=self.df_X.iloc[:,indices].columns.tolist()

        self.df_X=self.df_X.iloc[:,indices]
    # 3
    def remove_high_correlated_table(self,coorelation_threshold=0.8,pre_defined_kept_list=None,pre_defined_removed_list=None):

        df_coor_X=self.df_X.corr().abs()
        df_coor_X=df_coor_X.unstack().reset_index()
        df_coor_X.columns=['iv_1','iv_2','coor']
        df_coor_X_high=df_coor_X[df_coor_X['iv_1']!=df_coor_X['iv_2']]
        df_coor_X_high=df_coor_X_high[df_coor_X_high['coor']>coorelation_threshold]
        df_coor_X_high['high_coor_pairs']=df_coor_X_high[['iv_1','iv_2']].values.tolist()

        list_highly_pairs=df_coor_X_high['high_coor_pairs'].tolist()
        list_highly_pairs=[sorted(x) for x in list_highly_pairs]
        print("len(list_highly_pairs)",len(list_highly_pairs))

        list_highly_pairs=[str(x) for x in list_highly_pairs]
        list_highly_pairs=list(set(list_highly_pairs))
        list_highly_pairs=[eval(x) for x in list_highly_pairs]

        list_set_coor_unique=[]
        for x in list_highly_pairs:
            list_set_coor_unique=list_set_coor_unique+x
        list_set_coor_unique=list(set(list_set_coor_unique))
        if pre_defined_removed_list:
            set_kept=set(pre_defined_kept_list)
            set_removed=set(pre_defined_removed_list)
        else:
            print("we have %i pairs of highly correlated ivs, with %i unique values"%(len(list_highly_pairs),len(list_set_coor_unique)))
            print("pairs as below: \n%s"%str(list_highly_pairs))
            list_kept=[]
            list_removed=[]

            input_str=np.nan
            print("type in the col to put into kept(type 1 to move forward): ")
            while input_str!=str(1):
                input_str=input()
                try:                
                    list_set_coor_unique.remove(input_str)
                    list_kept.append(input_str)
                    print("list_kept for now: \n%s"%list_kept)
                    print("remaining unique elements: \n%s"%list_set_coor_unique)
                except:
                    print("moving next")


            print("type in the col to put into remove(type 2 to move forward): ")    
            while input_str!=str(2):
                input_str=input()
                try:                
                    list_set_coor_unique.remove(input_str)
                    list_removed.append(input_str)
                    print("removed for now: \n%s"%list_removed)
                    print("remaining unique elements: \n%s"%list_set_coor_unique)
                except:
                    print("moving next")

            set_kept=set(list_kept)
            set_removed=set(list_removed)
        set_common_cols=set_kept.intersection(set_removed)
        if set_common_cols:
            print("common col appared in both kept and remove: %s"%str(set_common_cols))
        else:
            self.X_features=[x for x in self.X_features if x not in set_removed]
            self.df_X=self.df_X[self.X_features]
        print("done of removing highly correlcated cols")
        print("cols removed: %s"%set_removed)
        
        # qc
        df_coor_X=self.df_X.corr().abs()
        df_coor_X=df_coor_X.unstack().reset_index()
        df_coor_X.columns=['iv_1','iv_2','coor']
        df_coor_X_high=df_coor_X[df_coor_X['iv_1']!=df_coor_X['iv_2']]
        df_coor_X_high=df_coor_X_high[df_coor_X_high['coor']>coorelation_threshold]
        if df_coor_X_high.shape[0]>0:
            print("still have high correlation above %s"%coorelation_threshold)
            
            
        self.removed_list_because_coor=list(set_removed)
        self.kept_list_because_coor=list(set_kept)
        
    # 4      
    def split_train_test(self,test_pctg=0.25):
        self.X_train, self.X_test, self.df_y_train, self.df_y_test = train_test_split(self.df_X, self.df_Y, test_size=test_pctg, random_state=42)

        self.X_train_scaled=scale(self.X_train)
        self.X_test_scaled=scale(self.X_test)
        # note: scale the X just to make the rfe faster
        print("X_train_scaled.shape",self.X_train_scaled.shape)
        print("X_test_scaled.shape",self.X_test_scaled.shape)

        print("X_train.shape",self.X_train.shape)
        print("X_test.shape",self.X_test.shape)
        
        
        self.input_y_train_list=self.df_y_train.iloc[:,self.n_week_DV-1].values.tolist()
        self.input_y_test_list=self.df_y_test.iloc[:,self.n_week_DV-1].values.tolist()
        
        
        list_index_y_train=list(self.df_y_train.index)
        self.list_ids_y_train=[self.list_all_ids[i] for i in list_index_y_train]
        list_index_y_test=list(self.df_y_test.index)
        self.list_ids_y_test=[self.list_all_ids[i] for i in list_index_y_test]
    # 5
    def run_updating_df_count(self):
        df_trian_X_count=pd.DataFrame({"row":self.X_train.shape[0],"col":self.X_train.shape[1]},index=["X_train"])
        df_test_X_count=pd.DataFrame({"row":self.X_test.shape[0],"col":self.X_test.shape[1]},index=["X_test"])
        self.db_row_counts=self.db_row_counts.append(df_trian_X_count)
        self.db_row_counts=self.db_row_counts.append(df_test_X_count)
    # 6    
    def generate_DV_distribution(self):
        df_y_train_count=pd.DataFrame()
        for col in self.df_y_train.columns.tolist():
            count_1=self.df_y_train[self.df_y_train[col]==1].shape[0]
            count_0=self.df_y_train[self.df_y_train[col]==0].shape[0]

            df=pd.DataFrame({"0":count_0,"1":count_1},index=[col])
            df_y_train_count=df_y_train_count.append(df)
        df_y_train_count.insert(0,"set","y_train")


        df_y_test_count=pd.DataFrame()
        for col in self.df_y_test.columns.tolist():
            count_1=self.df_y_test[self.df_y_test[col]==1].shape[0]
            count_0=self.df_y_test[self.df_y_test[col]==0].shape[0]

            df=pd.DataFrame({"0":count_0,"1":count_1},index=[col])
            df_y_test_count=df_y_test_count.append(df)
        df_y_test_count.insert(0,"set","y_test")

        self.df_y_train_count=df_y_train_count
        self.df_y_test_count=df_y_test_count
    # 7
    def select_from_model_n_features(self, N_feature_select_from_models=60):
        print("Starting select_from_model_n_features: ",datetime.datetime.now())
        selector = SelectFromModel(estimator=LogisticRegression(random_state=0,
                                                                solver="saga",
                                                                max_iter=2000,
                                                                n_jobs=24,
                                                                tol=0.0001),
                                   max_features=N_feature_select_from_models,
                                   threshold=-np.inf).fit(self.X_train_scaled, self.input_y_train_list)

        print("selector.threshold_",selector.threshold_)
        selector_support_FROMMODEL=selector.get_support()

        self.X_features=[self.X_features[i] for i,v in enumerate(selector_support_FROMMODEL) if v==True]

        self.X_train=self.X_train.loc[:,self.X_features]
        self.X_test=self.X_test.loc[:,self.X_features]

        print("X_train.shape",self.X_train.shape)
        print("X_test.shape",self.X_test.shape)
        print(datetime.datetime.now())

        self.X_train_scaled=scale(self.X_train)
        self.X_test_scaled=scale(self.X_test)
        # note: scale the X just to make the rfe faster
        print("X_train_scaled.shape",self.X_train_scaled.shape)
        print("X_test_scaled.shape",self.X_test_scaled.shape)
        print("Done select_from_model_n_features: ",datetime.datetime.now())
    # 8    
    def select_REF(self):
        print("Starting select_REF: ",datetime.datetime.now())

        estimator = LogisticRegression(fit_intercept=True,solver='saga',max_iter=2000,n_jobs=24,tol=0.001)
        selector = RFE(estimator,step=1,n_features_to_select=40)
        selector = selector.fit(self.X_train_scaled, self.input_y_train_list)
        selector_support_REF=selector.support_
        print("Done select_REF: ",datetime.datetime.now())

        self.X_features=[self.X_features[i] for i,v in enumerate(selector_support_REF) if v==True]

        self.X_train=self.X_train.loc[:,self.X_features]
        self.X_test=self.X_test.loc[:,self.X_features]

        print("X_train.shape",self.X_train.shape)
        print("X_test.shape",self.X_test.shape)
        self.X_train_scaled=scale(self.X_train)
        self.X_test_scaled=scale(self.X_test)
        print("X_train_scaled.shape",self.X_train_scaled.shape)
        print("X_test_scaled.shape",self.X_test_scaled.shape)
    # 9
    def forwards_feature_elimination_based_on_p_and_vif(self,niter=50,method="lbfgs",p_tol=0.1,vif_tol=5):
        len_x_features=self.X_train.shape[1]
        len_x_features_new=0
        df_x_dropped=self.X_train.copy()
        i_iter=0
        while len_x_features_new<len_x_features and i_iter<=100:
            i_iter+=1
            len_x_features=df_x_dropped.shape[1]
            mod=sm.Logit(self.input_y_train_list,sm.add_constant(df_x_dropped),niter=niter,method=method)
            res=mod.fit()
            table=res.summary2().tables[1]   
            X=add_constant(scale(df_x_dropped))
            list_cols=table.index.tolist()
            table["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]

            max_vif=table['VIF Factor'].max()
            max_p=table['P>|z|'].max()
            
            if max_vif>vif_tol:
                col_name_to_drop=table.index[table['VIF Factor']==max_vif][0]
                del df_x_dropped[col_name_to_drop]
                len_x_features_new=df_x_dropped.shape[1]
                print(df_x_dropped.shape,"column %s dropped due to high vif"%col_name_to_drop)
                
            elif max_p>p_tol:
                col_name_to_drop=table.index[table['P>|z|']==max_p][0]
                del df_x_dropped[col_name_to_drop]
                len_x_features_new=df_x_dropped.shape[1]
                print(df_x_dropped.shape,"column %s dropped due to p value"%col_name_to_drop)
            else:
                i_iter+=100
                

        self.X_train=df_x_dropped
        self.X_features=df_x_dropped.columns.tolist()
        self.X_test=self.X_test[self.X_features]
        self.X_train_scaled=scale(self.X_train)
        self.X_test_scaled=scale(self.X_test)
    # 10
    def run_sm_logR_model(self):
        self.sm_model=sm.Logit(self.input_y_train_list,sm.add_constant(self.X_train),niter=50,method="lbfgs")
        self.res_of_model=self.sm_model.fit()
        self.summary_table_over=self.res_of_model.summary2().tables[0].reset_index()
        self.summary_table_output=self.res_of_model.summary2().tables[1].reset_index()
        
        
        std=self.sm_model.exog.std(axis=0)
        std[0] = 1
        tt = self.res_of_model.t_test(np.diag(std))
        df_std_coef=tt.summary_frame()
        list_std_coefficients=df_std_coef['coef'].tolist()
        self.summary_table_output['std_coef']=list_std_coefficients
        
        self.list_train_pred=self.res_of_model.predict()
        self.list_test_pred=self.res_of_model.predict(sm.add_constant(self.X_test))
        
        coefficient_of_dermination = r2_score(self.input_y_train_list, self.list_train_pred)
        self.summary_table_over=self.summary_table_over.append(pd.DataFrame({"index":[8],0:"calculated_r_squared",1:coefficient_of_dermination},index=[8]))
    
        #VIF
        X=add_constant(self.X_train_scaled)
        list_cols=self.summary_table_output['index'].tolist()
        self.summary_table_output["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
        self.summary_table_output=self.summary_table_output.sort_values("std_coef")
    # 11
    def generate_step_table_of_test_SM(self,threshold_list = [(x+1)/100 for x in range(0,100)]):
        list_prob_test=self.res_of_model.predict(sm.add_constant(self.X_test))
        df_output=pd.DataFrame()
        for i in threshold_list:
            y_test_pred=[1 if x>i else 0 for x in list_prob_test]

            accuracy_score = metrics.accuracy_score(self.input_y_test_list,y_test_pred)    
            tn, fp, fn, tp = metrics.confusion_matrix(self.input_y_test_list, y_test_pred).ravel()
            # 
            TPR=tp/(tp+fn) #recall
            FNR=fn/(tp+fn)
            FPR=fp/(fp+tn)
            TNR=tn/(fp+tn)

            PPV=tp/(tp+fp) #precission
            f1_score = 2*(TPR*PPV)/(TPR+PPV)

            df=pd.DataFrame({"predicted_positive":len([x for x in y_test_pred if x==1]),
                             "predicted_negative":len([x for x in y_test_pred if x==0]),
                             "accuracy_score":accuracy_score,
                             'true_negative':tn,
                             'false_positive':fp,
                             'false_negative':fn,
                             'true_positive':tp,
                             'true_positive_rate':TPR,
                             'false_negative_rate':FNR,
                             'false_positive_rate':FPR,
                             'true_negative_rate':TNR,
                             'precission_(Positive predictive value)':PPV,
                             'f1_score':f1_score
                            },index=[i])
            df_output=df_output.append(df)
        
        self.df_step_table=df_output
    # 11-2
    def generate_step_table_of_test_SM_001_range(self,start_prob_pctg,end_prob_pctg,step=0.001):
        threshold_list = [(x+1)/1000 for x in range(int(start_prob_pctg*1000),int(end_prob_pctg*1000))]
        
        list_prob_test=self.res_of_model.predict(sm.add_constant(self.X_test))
        df_output=pd.DataFrame()
        for i in threshold_list:
            y_test_pred=[1 if x>i else 0 for x in list_prob_test]

            accuracy_score = metrics.accuracy_score(self.input_y_test_list,y_test_pred)    
            tn, fp, fn, tp = metrics.confusion_matrix(self.input_y_test_list, y_test_pred).ravel()
            # 
            TPR=tp/(tp+fn) #recall
            FNR=fn/(tp+fn)
            FPR=fp/(fp+tn)
            TNR=tn/(fp+tn)

            PPV=tp/(tp+fp) #precission
            f1_score = 2*(TPR*PPV)/(TPR+PPV)

            df=pd.DataFrame({"predicted_positive":len([x for x in y_test_pred if x==1]),
                             "predicted_negative":len([x for x in y_test_pred if x==0]),
                             "accuracy_score":accuracy_score,
                             'true_negative':tn,
                             'false_positive':fp,
                             'false_negative':fn,
                             'true_positive':tp,
                             'true_positive_rate':TPR,
                             'false_negative_rate':FNR,
                             'false_positive_rate':FPR,
                             'true_negative_rate':TNR,
                             'precission_(Positive predictive value)':PPV,
                             'f1_score':f1_score
                            },index=[i])
            df_output=df_output.append(df)
        
        self.df_step_table_2=df_output
        
    # 12
    def select_best_scored_pred_prob(self,defined_best_score=None):
        if defined_best_score is None:
            self.df_step_table['self_defined_score']=self.df_step_table.apply(lambda df: scoring_confusion_matrix(df['true_positive'],df['true_negative'],df['false_positive'],df['false_negative']),axis=1)
            threshold_max_selfdefinedscore=self.df_step_table[self.df_step_table['self_defined_score']==self.df_step_table['self_defined_score'].max()].index[0]
            self.threshold_max_selfdefinedscore=threshold_max_selfdefinedscore
            self.df_step_table=self.df_step_table.reset_index()
            self.df_confusion_table=self.df_step_table.loc[self.df_step_table['index']==threshold_max_selfdefinedscore,:]
        else:
            threshold_max_selfdefinedscore=defined_best_score
            self.threshold_max_selfdefinedscore=threshold_max_selfdefinedscore
            self.df_step_table_2=self.df_step_table_2.reset_index()
            self.df_confusion_table=self.df_step_table_2.loc[self.df_step_table_2['index']==threshold_max_selfdefinedscore,:]
            
    # 13
    def generate_gain_chart(self):
        self.df_gainchart_train=generate_gain_chart_function(df_X=self.X_train,
                                                             list_y=self.input_y_train_list,
                                                             list_ids=self.list_ids_y_train,
                                                             result_sm_model=self.res_of_model,
                                                             threshold=self.threshold_max_selfdefinedscore,
                                                             list_selected_features=self.X_features)
        
        self.df_gainchart_test=generate_gain_chart_function(df_X=self.X_test,
                                                            list_y=self.input_y_test_list,
                                                            list_ids=self.list_ids_y_test,
                                                            result_sm_model=self.res_of_model,
                                                            threshold=self.threshold_max_selfdefinedscore,
                                                            list_selected_features=self.X_features)
    def save_outputs(self):
        
        writer=pd.ExcelWriter(self.output_path,engine="xlsxwriter")
        
        self.db_row_counts.to_excel(writer,"df_dataset_shape")
        self.df_date_range.to_excel(writer,"df_date_range")
        self.df_y_train_count.to_excel(writer,"df_y_train_count")
        self.df_y_test_count.to_excel(writer,"df_y_test_count")
        self.summary_table_over.to_excel(writer,"summary_table_over")
        self.summary_table_output.to_excel(writer,"summary_table_output")
        self.df_step_table.to_excel(writer,"step_table",index=True)
        if self.df_step_table_2 is not None:
            self.df_step_table_2.to_excel(writer,"step_table_2",index=True)
        self.df_confusion_table.to_excel(writer,"select_score_matrix",index=False)
        
        self.df_gainchart_train.to_excel(writer,"gainchart_train",index=False)
        self.df_gainchart_test.to_excel(writer,"gainchart_test",index=False)
        self.df_department_name.to_excel(writer,"department_name",index=False)
        writer.save()
        
        

In [19]:
dict_config=json.load(open("/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/config.json"))
high_date=dict_config['pos_end_date']
username=dict_config['username']
password=dict_config['password']
database=dict_config['database']
recent_n_month=dict_config['recent_n_month']

dict_tables=json.load(open("/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/table_names_%s.json"%str(high_date).replace("-","")))
table_df_1=dict_tables['table_df_1']
table_2_1=dict_tables['table_2_1']
table_2_2=dict_tables['table_2_2']
table_0_train=dict_tables['table_crm_id_list_train']
table_0_test=dict_tables['table_crm_id_list_test']

BL_engine=sqlalchemy.create_engine("mysql+pymysql://%s:%s@localhost/%s" % (username, password, database))

In [3]:
for t in [table_df_1,table_2_1,table_2_2,table_0_train]:
    print(pd.read_sql("select count(*) from %s"%t,con=BL_engine).iloc[0],t)

/usr/local/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


count(*)    1975547
Name: 0, dtype: int64 table_pred_1_crm_up_to_20200704
count(*)    1975547
Name: 0, dtype: int64 all_NEall_id_pred_pos_2_1_pos_until_20200704
count(*)    1975547
Name: 0, dtype: int64 all_NEall_id_pred_pos_2_2_pos_until_20200704
count(*)    1000000
Name: 0, dtype: int64 crm_table_id_list_train_20200704


In [4]:
col_list_df0_train=pd.read_sql("desc %s"%table_0_train,con=BL_engine)['Field'].values.tolist()
col_list_df_1=pd.read_sql("desc %s"%table_df_1,con=BL_engine)['Field'].values.tolist()
col_list_2_1=pd.read_sql("desc %s"%table_2_1,con=BL_engine)['Field'].values.tolist()
col_list_2_2=pd.read_sql("desc %s"%table_2_2,con=BL_engine)['Field'].values.tolist()

In [5]:
cols_no_need=['sign_up_location','customer_zip_code','nearest_BL_store',
              'week_1st_trans','week_recent_one_trans','week_recent_two_trans']
for col_remove in cols_no_need:
    col_list_df_1=[x for x in col_list_df_1 if x != col_remove and x!="customer_id_hashed"]
    col_list_2_1=[x for x in col_list_2_1 if x != col_remove and x!="id"]
    col_list_2_2=[x for x in col_list_2_2 if x != col_remove and x!="id"]
    
sql_str_cols_df0_train=str(["t0."+x for x in col_list_df0_train]).replace("'","")[1:-1]  
sql_str_cols_df_1=str(["t1."+x for x in col_list_df_1]).replace("'","")[1:-1]
sql_str_cols_2_1=str(["t2_1."+x for x in col_list_2_1]).replace("'","")[1:-1]
sql_str_cols_2_2=str(["t2_2."+x for x in col_list_2_2]).replace("'","")[1:-1]
sql_str_cols_all=", ".join([sql_str_cols_df0_train,sql_str_cols_df_1,sql_str_cols_2_1,sql_str_cols_2_2])
# sql_str_cols_all

In [6]:
queary="SELECT %s from %s as t0 \
left join %s as t1 on t0.customer_id_hashed=t1.customer_id_hashed \
left join %s as t2_1 on t0.customer_id_hashed=t2_1.id \
left join %s as t2_2 on t0.customer_id_hashed=t2_2.id"%(sql_str_cols_all,table_0_train,table_df_1,table_2_1,table_2_2)
# queary

In [7]:
print(datetime.datetime.now())
df_train=pd.read_sql(queary,con=BL_engine)
print(df_train.shape)
print(datetime.datetime.now())

2020-08-13 15:57:24.063688
(1000000, 249)
2020-08-13 16:10:34.706826


In [8]:
list_cols=df_train.columns.tolist()

for col in list_cols:
    if df_train[col].nunique()==1:
        del df_train[col]
        print("col_nunique==1 dropped: %s"%col)

col_nunique==1 dropped: department_115_trans
col_nunique==1 dropped: department_115_1st_trans
col_nunique==1 dropped: department_540_1st_trans
col_nunique==1 dropped: department_115_recent_one
col_nunique==1 dropped: department_540_recent_one
col_nunique==1 dropped: department_115_recent_two


In [10]:
print(datetime.datetime.now())
df_train_copy=df_train.head(3*10**5)
print(df_train_copy.shape)
df_train_copy=df_train_copy.T.drop_duplicates().T
print(datetime.datetime.now())
print(df_train_copy.shape)

list_cols_keep=df_train_copy.columns.tolist()
df_train=df_train[list_cols_keep]
print(df_train.shape)

del df_train_copy
gc.collect()

2020-08-13 16:52:22.396399
(300000, 243)
2020-08-13 16:59:15.046820
(300000, 240)
(1000000, 240)


0

0.23.1


In [24]:
# iv_start_date
iv_end_date=datetime.datetime.strptime(high_date,"%Y-%m-%d").date()
iv_start_date=iv_end_date-datetime.timedelta(days=int(np.ceil(365*recent_n_month/12)))

dv_end_date=iv_end_date+datetime.timedelta(days=28)
dv_start_date=iv_end_date+datetime.timedelta(days=1)  
df_date_range=pd.DataFrame({"start":[iv_start_date,dv_start_date],"end":[iv_end_date,dv_end_date]},index=['IVs',"DVs"])

In [27]:
def scoring_confusion_matrix(tp,tn,fp,fn):
    total=sum([tp,tn,fp,fn])
    accuracy=(tp+tn)/total
    ppv=tp/(tp+fp) # positive predict value
    fdr=fp/(tp+fp) # false discover rate
    fpr=fp/(tn+fp) # false positive rate
    
    TPR=tp/(tp+fn) #recall
    PPV=tp/(tp+fp) #precission
    f1_score = 2*(TPR*PPV)/(TPR+PPV)
    
    score=9*tp-8*fn*0.5-fp
    # the score ignored the f1 and overall accuracy due to low pctg
    
    
    
    # consider the profit vs lost 10:1 (30%*$30) vs (cpc*frequecy or click)
    # which means 1 missed (fn) is 10 times of 1 wrong targeted (fp)
    # very aribitury
    return score

def write_aggregate_func_gain_chart(list_selected_features,df_pred_table_detail):
    func_dict={"customer_id_hashed":"count"}
    list_cols_for_ratios=['y_true','y_hat']
    for col in list_selected_features:
        if len(df_pred_table_detail[col].unique())==2:
            func_dict.update({col:'sum'})
            list_cols_for_ratios.append(col)
        else:
            func_dict.update({col:"mean"})
    func_dict.update({"y_true":"sum"})
    func_dict.update({"y_hat":"sum"})
    # func_dict.update({"pred_prob":['max','min']})
    return func_dict,list_cols_for_ratios


def generate_gain_chart_function(df_X,list_y,list_ids,result_sm_model,threshold,list_selected_features):
    list_pred_prob=result_sm_model.predict(sm.add_constant(df_X)).values
    df_pred_by_id=pd.DataFrame({"customer_id_hashed":list_ids,"pred_prob":list_pred_prob},index=range(len(list_pred_prob)))
    copy_xtrain=df_X.copy().reset_index()
    del copy_xtrain['index']
    df_pred_by_id=pd.concat([df_pred_by_id,copy_xtrain],axis=1,ignore_index=False)

    df_pred_by_id['decile']=pd.qcut(df_pred_by_id['pred_prob'], 10, labels=False)
    df_pred_by_id['decile']=df_pred_by_id['decile'].apply(lambda x: "D"+str(x+1).zfill(2))
    df_pred_by_id['y_true']=list_y
    df_pred_by_id['y_hat']=np.where(df_pred_by_id['pred_prob']>threshold,1,0)

    agg_func,list_cols_to_get_ratio=write_aggregate_func_gain_chart(list_selected_features,df_pred_by_id)
    df_gainchart=df_pred_by_id.groupby("decile")[['customer_id_hashed']+list_selected_features].agg(agg_func).reset_index()

    df_prob_max=df_pred_by_id.groupby("decile")['pred_prob'].max().to_frame().reset_index().rename(columns={"pred_prob":"max_prob"})
    df_prob_min=df_pred_by_id.groupby("decile")['pred_prob'].min().to_frame().reset_index().rename(columns={"pred_prob":"min_prob"})
    df_gainchart=pd.merge(df_gainchart,df_prob_max,on="decile")
    df_gainchart=pd.merge(df_gainchart,df_prob_min,on="decile")
    df_gainchart.insert(2,"actual_ratio",df_gainchart['y_true']/df_gainchart['customer_id_hashed'])
    df_gainchart.insert(3,"pred_ratio",df_gainchart['y_hat']/df_gainchart['customer_id_hashed'])


    df_gainchart.insert(4,"max_pred_prob",df_gainchart['max_prob'])
    df_gainchart.insert(5,"min_pred_prob",df_gainchart['min_prob'])
    del df_gainchart['max_prob']
    del df_gainchart['min_prob']

    return df_gainchart